## Transform Orders Data- Extracting from Json Object
<ol>
<li>Access Elements from Json Object</li>
<li>Deduplicate Array elements</li>
<li>Explode Arrays</li>
<li>Write Transformed data to silver schema</li>
</ol>


In [0]:
select * from gizmobox_cat.silver.orders_json;

###1.Access Elements from Json Object

In [0]:
select json_value.order_id,json_value.order_status,
json_value.payment_method,json_value.total_amount,
json_value.transaction_timestamp,json_value.customer_id,json_value.items
 from gizmobox_cat.silver.orders_json;

###2.Deduplicate Array elements
<a href='https://docs.databricks.com/gcp/en/sql/language-manual/functions/array_distinct'>Array_Distinct Function</a>

In [0]:
select json_value.order_id,json_value.order_status,
json_value.payment_method,json_value.total_amount,
json_value.transaction_timestamp,json_value.customer_id,
array_distinct(json_value.items) as items
 from gizmobox_cat.silver.orders_json;

### 3.Explode Arrays

In [0]:
CREATE OR REPLACE TEMPORARY VIEW tv_orders_exploded as
select json_value.order_id,json_value.order_status,
json_value.payment_method,json_value.total_amount,
json_value.transaction_timestamp,json_value.customer_id,
explode(array_distinct(json_value.items)) as item
 from gizmobox_cat.silver.orders_json;

In [0]:
select order_id,order_status,payment_method,total_amount,transaction_timestamp
,customer_id,
item.item_id,item.name,item.price,item.quantity,item.category,item.details.brand,item.details.color from tv_orders_exploded;

### 4.Write Transformed data to silver schema

In [0]:
CREATE TABLE IF NOT EXISTS gizmobox_cat.silver.orders as
select order_id,order_status,payment_method,total_amount,transaction_timestamp
,customer_id,
item.item_id,item.name,item.price,item.quantity,item.category,item.details.brand,item.details.color from tv_orders_exploded;

In [0]:
select * from gizmobox_cat.silver.orders;